In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
%matplotlib inline

import sys
import os
import yaml
import itertools
sys.path.append('../')
from utils.utils import ModelLoader,DepLoader
from matplotlib import pyplot as plt
from timeit import timeit  

In [125]:
with open('../config/rule_shortdeck.yaml') as fhdl:
    cfg = yaml.load(fhdl)

/root/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [126]:
env = DepLoader(cfg)

In [127]:
depth = 6
json = env.tree_builder.gen_km_json("../imgs/tree_depth{}.km".format(depth),limit=depth,ret_json=True)

# 查看游戏总状态数

In [128]:
hiss = []
roots = []
def count_dealcard(root,depth=0):
    if root.children is not None:
        num = 0
        for one_child in root.children.values():
            num += count_dealcard(one_child)
        return num
    else:
        if 'terminal' in root.to_string():
            hiss.append(root.bet_history)
            roots.append(root)
        return 0 if 'terminal' in root.to_string() else 1

In [129]:
count_dealcard(env.tree_builder.root)

47662

In [7]:
%timeit count_dealcard(env.tree_builder.root)

891 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%lprun -f count_dealcard count_dealcard(env.tree_builder.root)

Timer unit: 1e-06 s

Total time: 1.82193 s
File: <ipython-input-5-3dbdc1fb9032>
Function: count_dealcard at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def count_dealcard(root,depth=0):
     4    156709        83402      0.5      4.6      if root.children is not None:
     5     61385        23555      0.4      1.3          num = 0
     6    218093       115805      0.5      6.4          for one_child in root.children.values():
     7    156708        76103      0.5      4.2              num += count_dealcard(one_child)
     8     61385        22854      0.4      1.3          return num
     9                                               else:
    10     95324       751080      7.9     41.2          if 'terminal' in root.to_string():
    11     47662        28820      0.6      1.6              hiss.append(root.bet_history)
    12     47662        23456      0.5      1.3              roots.append(root)
    13  

In [9]:
len(hiss)

476620

# 查看叶子节点的bet history

In [10]:
index = 5000 + 1
print(hiss[index])
print(roots[index].committed)

['call', 'check', 'check', 'bet_1', 'raise_2', 'raise_5', 'call', 'check', 'bet_7', 'raise_21', 'raise_84', 'call', 'check', 'bet_2', 'fold']
[100.0, 98.0]


# 查看card abstraction 工作状态

In [18]:
cards_sampled = env.card_sampler.sample_card(4)
print(cards_sampled)

['Qc', 'Ah', '6s', 'As']


In [19]:
env.card_abstraction.abstract(cards_sampled)

'QcAh6sAs'

In [20]:
# TODO 写strategy 和trainer,修改abstraction算法
# TODO 每层都需要用不同的action choice

In [148]:
from cython.parallel import prange, parallel, threadid
import time
import multiprocessing

In [170]:
import utils.utils as utils
pb = utils.ProgressBar(worksum=5)
pb.startjob()

def func(msg):
    time.sleep(3)
    pb.complete(1)
    return msg

In [171]:
t1 = time.time()
pool = multiprocessing.Pool(processes = 3)
results = []
for i in range(5):
    msg = "hello %d" %(i)
    one_res = pool.apply_async(func, (msg, ))   #维持执行的进程总数为processes，当一个进程执行完毕后会添加新的进程进去
    results.append(one_res)
print ("Mark~ Mark~ Mark~~~~~~~~~~~~~~~~~~~~~~")
pool.close()
pool.join()   #调用join之前，先调用close函数，否则会出错。执行完close后不会有新的进程加入到pool,join函数等待所有子进程结束
print("Sub-process(es) done.")
for i in range(len(results)):
    results[i] = results[i].get()

print(time.time() - t1)

Mark~ Mark~ Mark~~~~~~~~~~~~~~~~~~~~~~
 20.00 % [==========>----------------------------------------] 1/5 	 used:3s eta:12 sSub-process(es) done.
6.151718616485596


In [159]:
alljson

{}

In [152]:
t1 = time.time()
for i in range(4):
    msg = "hello %d" %(i)
    func(msg)   #维持执行的进程总数为processes，当一个进程执行完毕后会添加新的进程进去
print ("Mark~ Mark~ Mark~~~~~~~~~~~~~~~~~~~~~~")
print("Sub-process(es) done.")

print(time.time() - t1)

msg: hello 0
end
msg: hello 1
end
msg: hello 2
end
msg: hello 3
end
Mark~ Mark~ Mark~~~~~~~~~~~~~~~~~~~~~~
Sub-process(es) done.
12.016242265701294
